In [17]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from datasets.img_dataset import FoodImgs
from models.cnn import SnapSnack

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
import math


dataset = FoodImgs()

s = len(dataset)
print(f"{s} imgs")
train_size = int(math.ceil(s * 0.8))
test_size = s - train_size

train_set, test_set = random_split(dataset, [train_size, test_size])


9090 imgs


In [ ]:
from pytorch_lightning.loggers import WandbLogger
import wandb
# wandb.init(project="snapnsnack")
wandb_logger = WandbLogger(project='snapnsnack')
model = SnapSnack(output_dim=4)
trainer = pl.Trainer(logger=wandb_logger)
trainer.fit(model, DataLoader(train_set, batch_size=1), DataLoader(test_set))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping validation loop
  warnings.warn(*args, **kwargs)


In [ ]:
import torch
import pickle as pkl
import pandas as pd
epi = pd.read_csv("../data/snapnsnackdb/epi_r.csv")

epi.head()

In [30]:
img_4 = "/Users/davinan/Dropbox/github/Snap-N-Snack/data/snapnsnackdb/simple_images/Spinach+Noodle+Casserole+/Spinach+Noodle+Casserole+_1.jpeg"

In [35]:
import matplotlib.pyplot as plt
im = plt.imread(img_4)

In [43]:
from torchvision import transforms as T
from PIL import Image

transform = T.Compose(
            [
                T.ToTensor(),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
                T.Resize(256),
                T.CenterCrop(224),
            ]
        )

model.eval()
oi = transform(Image.fromarray(im)).unsqueeze(0)
preds = model(oi)

In [44]:
preds

tensor([[  3.7612, 327.7388,  11.6335,  22.7230, 274.9613]],
       grad_fn=<AddmmBackward>)

In [5]:
import torch
torch.__version__

'1.7.0'

In [10]:
import torchvision
torchvision.__version__

'0.8.1'